DVD Rental Duration Prediction Project
----

**Project Task**

A DVD rental company wants to predict how many days a customer will keep a rented DVD.
They have provided a dataset (`rental_info.csv`) with customer rental information and request that you:

1. **Build regression models** to predict the rental duration (in days).
2. **Preprocess the data**:

   * Convert `rental_date` and `return_date` to datetime format.
   * Calculate `rental_length_days` as the difference between return and rental dates.
   * Create dummy variables for specific `special_features` (e.g., Deleted Scenes, Behind the Scenes).
   * Drop unnecessary original columns.
3. **Perform feature selection** using Lasso Regression.
4. **Train and evaluate** the following models:

   * Linear Regression (with Lasso-selected features)
   * Decision Tree Regressor (all features)
   * Random Forest Regressor (all features, with hyperparameter tuning via `RandomizedSearchCV`)
5. **Measure performance** using Mean Squared Error (MSE) on the test set.
6. **Select the best model**:

   * The target is an MSE of **3 or less**.
   * Print the MSE of each model and identify the best performer.

**Dataset Features**
The dataset contains:

* Dates: `rental_date`, `return_date`
* Payment info: `amount`, `amount_2`
* Rental rate info: `rental_rate`, `rental_rate_2`
* Movie details: `release_year`, `length`, `length_2`, `replacement_cost`, `special_features`
* Rating dummies: `NC-17`, `PG`, `PG-13`, `R`

The target variable is `rental_length_days`.

---

Do you want me to also rewrite it in a **short, single-paragraph format** so it’s easier to paste into a README?


In [2]:
#Importing necessary libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error


In [3]:
rental_df = pd.read_csv(r"C:\Users\Emigb\Documents\Data Science\datasets\rental_info.csv")
rental_df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [4]:
rental_df.isna().sum()

rental_date         0
return_date         0
amount              0
release_year        0
rental_rate         0
length              0
replacement_cost    0
special_features    0
NC-17               0
PG                  0
PG-13               0
R                   0
amount_2            0
length_2            0
rental_rate_2       0
dtype: int64

In [5]:
rental_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


Data Preprocessing
----

Convert `rental_date` and `return_date` to datetime format.

In [6]:
#Converting columns to DateTime
rental_df['rental_date'] = pd.to_datetime(rental_df['rental_date'])
rental_df['return_date'] = pd.to_datetime(rental_df['return_date'])

#Checking Datatypes to confirm change
print(rental_df['return_date'].dtypes)
print(rental_df['rental_date'].dtypes)

datetime64[ns, UTC]
datetime64[ns, UTC]


Calculate `rental_length_days` as the difference between return and rental dates.

In [7]:
rental_df['rental_length_days'] = (rental_df['return_date'] - rental_df['rental_date']).dt.days

rental_df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4


Create dummy variables for specific `special_features` (e.g., Deleted Scenes, Behind the Scenes)

In [8]:
rental_df["deleted_scenes"] = np.where(rental_df["special_features"].str.contains("Deleted Scenes"), 1, 0)
rental_df['behind_the_scene'] = np.where(rental_df['special_features'].str.contains("Behind the Scenes"), 1,0)
rental_df.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,deleted_scenes,behind_the_scene
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,3,0,1
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2,0,1
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,7,0,1
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,2,0,1
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401,4,0,1


In [9]:
rental_df.shape

(15861, 18)

Drop unnecessary original columns.

In [10]:
rental_df.drop(columns = ['rental_date', 'return_date', 'special_features'], inplace=True)
rental_df.head()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2,rental_length_days,deleted_scenes,behind_the_scene
0,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,3,0,1
1,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,2,0,1
2,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,7,0,1
3,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,2,0,1
4,2.99,2005.0,2.99,126.0,16.99,0,0,0,1,8.9401,15876.0,8.9401,4,0,1


Feature Selection
----

In [11]:
X = rental_df.drop('rental_length_days', axis=1)
y = rental_df['rental_length_days']

Splitting Data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=9)

Perform feature selection using Lasso Regression.

In [13]:
lasso = Lasso(alpha = 0.1, random_state=9)
lasso.fit(X_train, y_train)

lasso_coef = lasso.coef_
X_train_laso = X_train.iloc[:, lasso_coef > 0]
X_test_lasso = X_test.iloc[:, lasso_coef > 0]

Training & Evaluation
----

Linear Regression

In [16]:
lr = LinearRegression()
lr.fit(X_train_laso, y_train)
y_pred_lr = lr.predict(X_test_lasso)
MSE_lr = mean_squared_error(y_test, y_pred_lr)

Decision Tree

In [17]:
dt = DecisionTreeRegressor(random_state=9)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
MSE_dt = mean_squared_error(y_test, y_pred_dt)

Random Forest

In [18]:
param_dist = {'n_estimators':np.arange(1,101,1),
              'max_depth':np.arange(1,11,1)}
rf = RandomForestRegressor()
rand_search = RandomizedSearchCV(rf,
                                 param_distributions = param_dist,
                                 cv = 5,
                                 random_state=9)

rand_search.fit(X_train, y_train)
hyper_params = rand_search.best_params_

In [19]:
rf = RandomForestRegressor(n_estimators=hyper_params['n_estimators'],
                           max_depth = hyper_params['max_depth'],
                           random_state = 9)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
MSE_rf = mean_squared_error(y_test, y_pred_rf)

Select the best model
----

In [20]:
results = {
    "Linear Regression": MSE_lr,
    "DecisionTree": MSE_dt,
    "RandomForest": MSE_rf
}

In [21]:
# Select best model
best_model_name = min(results, key=results.get)
best_model_mse = results[best_model_name]

print("MSE Results:", results)
print(f"Best Model: {best_model_name} with MSE = {best_model_mse}")

MSE Results: {'Linear Regression': 4.812297241276241, 'DecisionTree': 2.1675004952579413, 'RandomForest': 2.225667528098759}
Best Model: DecisionTree with MSE = 2.1675004952579413
